In [1]:
from __future__ import print_function
from openpathsampling.experimental.storage import Storage, ops_class_info, ops_schema
from openpathsampling.experimental.simstore import SQLStorageBackend
import numpy as np
from openpathsampling.experimental.simstore.serialization_helpers import get_uuid, set_uuid

import openpathsampling as paths

import collections

from openpathsampling.experimental.simstore.storable_functions import (
    StorableFunction, StorageFunctionHandler
)

In [2]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)

## Playing around with a storable function

In [3]:
def get_x(snap):
    from time import sleep
    sleep(2)
    return snap.xyz[0][0]

In [4]:
cv = StorableFunction(get_x)

In [5]:
import openpathsampling.engines.toy as toys
snap = toys.Snapshot(coordinates=np.array([[3.0, 0.0]]))

In [6]:
%%time
cv(snap)

2021-07-22 12:05:33,445 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:33,452 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:33,454 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
CPU times: user 5.06 ms, sys: 2.39 ms, total: 7.45 ms
Wall time: 2.01 s


3.0

In [7]:
%%time
cv(snap)

2021-07-22 12:05:35,477 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
CPU times: user 1.29 ms, sys: 1.37 ms, total: 2.66 ms
Wall time: 6.12 ms


3.0

## Manually registering storable function with SQL backend

In [8]:
backend = SQLStorageBackend("test.sql", mode='w', echo=False)
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info,
)

2021-07-22 12:05:35,533 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2021-07-22 12:05:35,535 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2021-07-22 12:05:35,537 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sfr_result_types
2021-07-22 12:05:35,540 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tags
2021-07-22 12:05:35,579 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2021-07-22 12:05:35,593 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2021-07-22 12:05:35,608 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2021-07-22 12:05:35,619 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2021-07-22 12:05:35,631 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema

In [9]:
uuid = get_uuid(cv)

In [10]:
_ = cv(snap)
cv.local_cache.result_dict

2021-07-22 12:05:35,855 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>


{'218455970078904451893013535049010642960': 3.0}

In [11]:
from openpathsampling.experimental.simstore.attribute_handlers import StandardHandler
backend.serialization['float'] = StandardHandler('float')

backend.register_storable_function(table_name=uuid, result_type='float')

2021-07-22 12:05:35,881 - openpathsampling.experimental.simstore.sql_backend - INFO - Registering storable function: UUID: 218455970078904451893013535049010642956 (float)


In [12]:
backend.add_storable_function_results(table_name=uuid,
                                      result_dict=cv.local_cache.result_dict)

2021-07-22 12:05:35,908 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [13]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2021-07-22 12:05:35,936 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'218455970078904451893013535049010642960': 3.0}

In [14]:
backend.load_storable_function_table(get_uuid(cv))

{'218455970078904451893013535049010642960': 3.0}

## New backend; use storable func with storage

This will be roughly the full integration.

In [15]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2021-07-22 12:05:35,976 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2021-07-22 12:05:35,978 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2021-07-22 12:05:35,982 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sfr_result_types
2021-07-22 12:05:35,991 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tags
2021-07-22 12:05:36,017 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2021-07-22 12:05:36,045 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2021-07-22 12:05:36,061 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2021-07-22 12:05:36,072 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2021-07-22 12:05:36,089 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema

In [16]:
backend.identifier

('sqlite:///test.sql', 'w')

In [17]:
storage.save(cv)

2021-07-22 12:05:36,250 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:36,257 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:36,260 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:36,266 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-07-22 12:05:36,268 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-07-22 12:05:36,271 - openpathsampling.experimental.simstore.storage - DEBUG - Found 2 objects
2021-07-22 12:05:36,273 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-07-22 12:05:36,275 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-07-22 12:05:36,277 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-07-22 12:05:36,279 - openpathsampling.experi

In [18]:
list(storage.simulation_objects)

[]

In [19]:
backend.metadata.tables.keys()

dict_keys(['schema', 'metadata', 'uuid', 'tables', 'sfr_result_types', 'tags', 'samples', 'sample_sets', 'trajectories', 'move_changes', 'steps', 'details', 'storable_functions', 'simulation_objects', 'storage_objects', '218455970078904451893013535049010642956'])

In [20]:
storage._sf_handler.functions[0].local_cache.result_dict

{'218455970078904451893013535049010642960': 3.0}

In [21]:
backend.load_storable_function_results(uuid, [get_uuid(snap)])

2021-07-22 12:05:36,477 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


{'218455970078904451893013535049010642960': 3.0}

In [22]:
storage.close()

In [23]:
backend = SQLStorageBackend("test.sql", mode='r')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2021-07-22 12:05:36,614 - openpathsampling.experimental.simstore.storage - INFO - Missing info from 0 dynamically-registered tables
2021-07-22 12:05:36,615 - openpathsampling.experimental.storage.ops_storage - INFO - Found 0 possible lookups
2021-07-22 12:05:36,616 - openpathsampling.experimental.storage.ops_storage - INFO - Lookups for tables: dict_keys([])
2021-07-22 12:05:36,629 - openpathsampling.experimental.simstore.storage - INFO - Successfully registered 0 missing tables
2021-07-22 12:05:36,646 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-07-22 12:05:36,647 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 1 non-cached objects
2021-07-22 12:05:36,648 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:36,651 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:36,655 - openpathsampling.experimental.s

In [24]:
cv_reloaded = storage.load([get_uuid(cv)])[0]

2021-07-22 12:05:36,743 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 1 objects
2021-07-22 12:05:36,745 - openpathsampling.experimental.simstore.storage - DEBUG - Getting internal structure of 0 non-cached objects
2021-07-22 12:05:36,748 - openpathsampling.experimental.simstore.storage - DEBUG - Loading 0 objects; creating 0 lazy proxies
2021-07-22 12:05:36,750 - openpathsampling.experimental.simstore.storage - DEBUG - Identifying classes for 0 lazy proxies
2021-07-22 12:05:36,752 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 0 UUIDs
2021-07-22 12:05:36,754 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 0 UUIDs
2021-07-22 12:05:36,767 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-07-22 12:05:36,768 - openpathsampling.experimental.simstore.storage - DEBUG - Reconstructing from 0 objects


In [25]:
cv_reloaded.local_cache.result_dict

{}

In [26]:
cv_reloaded.local_cache != cv.local_cache

True

In [27]:
%%time
cv_reloaded.local_cache.clear()
cv_reloaded(snap)

2021-07-22 12:05:36,806 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df56c810>>
2021-07-22 12:05:36,809 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df56c810>>
2021-07-22 12:05:36,821 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
CPU times: user 8.08 ms, sys: 2.91 ms, total: 11 ms
Wall time: 15.8 ms


3.0

In [28]:
# TODO: tests need a test of a bad load as well (missing should show up)

In [29]:
cv_reloaded.local_cache.clear()
cv_reloaded.preload_cache()

In [30]:
%%time
cv_reloaded(snap)

2021-07-22 12:05:36,872 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df56c810>>
CPU times: user 1.37 ms, sys: 1.47 ms, total: 2.84 ms
Wall time: 2.84 ms


3.0

## Store/load multiple values at once

This checks that the backend is able to load multiple UUIDs at once

In [31]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2021-07-22 12:05:36,907 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2021-07-22 12:05:36,908 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2021-07-22 12:05:36,910 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sfr_result_types
2021-07-22 12:05:36,915 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tags
2021-07-22 12:05:36,939 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2021-07-22 12:05:36,954 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2021-07-22 12:05:36,961 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2021-07-22 12:05:36,969 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2021-07-22 12:05:36,977 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema

In [32]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))

In [33]:
%%time
cv.local_cache.clear()
cv._handler = None
cv([snap, new_snap])

2021-07-22 12:05:37,188 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:37,190 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:37,191 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
CPU times: user 5.2 ms, sys: 2.37 ms, total: 7.57 ms
Wall time: 4.01 s


[3.0, 2.0]

In [34]:
%%time
# get using cache
cv([snap, new_snap])

2021-07-22 12:05:41,211 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
CPU times: user 1.31 ms, sys: 948 µs, total: 2.26 ms
Wall time: 2.6 ms


[3.0, 2.0]

In [35]:
storage.save(cv)

2021-07-22 12:05:41,223 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:41,230 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:41,231 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:41,235 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-07-22 12:05:41,251 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-07-22 12:05:41,256 - openpathsampling.experimental.simstore.storage - DEBUG - Found 2 objects
2021-07-22 12:05:41,258 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-07-22 12:05:41,261 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-07-22 12:05:41,262 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-07-22 12:05:41,263 - openpathsampling.experi

In [36]:
%%time
# get using storage
cv.local_cache.clear()
cv([snap, new_snap])

2021-07-22 12:05:41,424 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:41,426 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df3c0750>>
2021-07-22 12:05:41,429 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 2 UUIDs
CPU times: user 4.67 ms, sys: 2.31 ms, total: 6.98 ms
Wall time: 6.56 ms


[3.0, 2.0]

## Save multiple times

The second time we save things, the `find_uuid` skips over the already-saved function. This means that we need another mechanism for storing mutable information (such as the `StorableFunctionResults`) to disk.

This is part of what the `StorableFunctionHandler` manages. Since the handler contains the function, it can store things to disk with an update.

In [37]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)
cv = StorableFunction(get_x)
vol = paths.CVDefinedVolume(cv, lambda_min=0.0, lambda_max=1.0)
cv(snap)
storage.save(vol)

2021-07-22 12:05:41,451 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2021-07-22 12:05:41,453 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2021-07-22 12:05:41,456 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sfr_result_types
2021-07-22 12:05:41,458 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tags
2021-07-22 12:05:41,478 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2021-07-22 12:05:41,490 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2021-07-22 12:05:41,512 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2021-07-22 12:05:41,531 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2021-07-22 12:05:41,540 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema

In [38]:
new_snap = toys.Snapshot(coordinates=np.array([[2.0, 0.0]]))
cv(new_snap)

2021-07-22 12:05:43,917 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df654d10>>
2021-07-22 12:05:43,919 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df654d10>>
2021-07-22 12:05:43,958 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-07-22 12:05:43,964 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df654d10>>


2.0

In [39]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'218455970078904451893013535049010642960': 3.0}

In [40]:
storage.save(vol)

2021-07-22 12:05:45,997 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:46,001 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:46,002 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:46,012 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs


In [41]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'218455970078904451893013535049010642960': 3.0}

In [42]:
storage.save_function_results()

2021-07-22 12:05:46,084 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs


In [43]:
storage.backend.load_storable_function_table(get_uuid(cv))

{'218455970078904451893013535049010642960': 3.0,
 '218455970078904451893013535049010642994': 2.0}

In [44]:
list(storage._sf_handler.canonical_functions.values())[0].local_cache.result_dict

{'218455970078904451893013535049010642960': 3.0,
 '218455970078904451893013535049010642994': 2.0}

## Mock of multiple computers creating and returning results

In [45]:
cv = StorableFunction(get_x)
cv(snap)

2021-07-22 12:05:46,178 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cc190>>
2021-07-22 12:05:46,179 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cc190>>
2021-07-22 12:05:46,180 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cc190>>


3.0

### Computer 1

We mock the serialization and transfer over the network by doing a `to_dict`/`from_dict` cycle, which will create a new instance in memory with the same UUID.

In [46]:
cv_1 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_1, get_uuid(cv))

In [47]:
print("\n".join([repr(cv), repr(cv_1)]))

In [48]:
cv_1.local_cache.result_dict

{}

In [49]:
snap_1 = toys.Snapshot(coordinates=np.array([[4.0, 0.0]]))

In [50]:
%%time
cv_1(snap_1)

2021-07-22 12:05:48,290 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6ccc50>>
2021-07-22 12:05:48,295 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6ccc50>>
2021-07-22 12:05:48,300 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6ccc50>>
CPU times: user 5.79 ms, sys: 2.66 ms, total: 8.45 ms
Wall time: 2.02 s


4.0

In [51]:
cv_1.local_cache.result_dict

{'218455970078904451893013535049010643004': 4.0}

### Computer 2

In [52]:
cv_2 = StorableFunction.from_dict(cv.to_dict())
set_uuid(cv_2, get_uuid(cv))

In [53]:
print("\n".join([repr(cv), repr(cv_1), repr(cv_2)]))

In [54]:
snap_2 = toys.Snapshot(coordinates=np.array([[5.0, 0.0]]))

In [55]:
cv_2(snap_2)

2021-07-22 12:05:50,393 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_cached of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cbd10>>
2021-07-22 12:05:50,399 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._get_storage of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cbd10>>
2021-07-22 12:05:50,404 - openpathsampling.experimental.simstore.storable_functions - DEBUG - <bound method StorableFunction._eval of <openpathsampling.experimental.simstore.storable_functions.StorableFunction object at 0x7f85df6cbd10>>


5.0

In [56]:
cv_2.local_cache.result_dict

{'218455970078904451893013535049010643010': 5.0}

### Gather operation

Note that for the real thing, we'll need to ensure that the serializers/deserializers (be they from `dask` or custom) correctly recreate the objects.

Attention: The save command must be issued separately for each copy of the CV. This is because the CVs have the same UUID, and we use sets/dicts to combine repeats of the same UUID.

In [57]:
backend = SQLStorageBackend("test.sql", mode='w')
storage = Storage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

2021-07-22 12:05:52,447 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table uuid
2021-07-22 12:05:52,456 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tables
2021-07-22 12:05:52,463 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sfr_result_types
2021-07-22 12:05:52,470 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table tags
2021-07-22 12:05:52,497 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table samples
2021-07-22 12:05:52,518 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table sample_sets
2021-07-22 12:05:52,535 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table trajectories
2021-07-22 12:05:52,558 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema table move_changes
2021-07-22 12:05:52,590 - openpathsampling.experimental.simstore.sql_backend - INFO - Add schema

In [58]:
storage.save([cv])

2021-07-22 12:05:52,797 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:52,801 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:52,821 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:52,830 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 0 UUIDs
2021-07-22 12:05:52,833 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-07-22 12:05:52,835 - openpathsampling.experimental.simstore.storage - DEBUG - Found 2 objects
2021-07-22 12:05:52,837 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-07-22 12:05:52,839 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-07-22 12:05:52,841 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-07-22 12:05:52,844 - openpathsampling.experi

In [59]:
storage.save([cv_1])

2021-07-22 12:05:52,960 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:52,963 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:52,964 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:52,972 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2021-07-22 12:05:52,974 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-07-22 12:05:52,975 - openpathsampling.experimental.simstore.storage - DEBUG - Found 1 objects
2021-07-22 12:05:52,978 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-07-22 12:05:52,979 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-07-22 12:05:52,981 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-07-22 12:05:52,982 - openpathsampling.experi

In [60]:
storage.save([cv_2])

2021-07-22 12:05:53,060 - openpathsampling.experimental.simstore.storage - DEBUG - Starting save
2021-07-22 12:05:53,063 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Looking for 1 UUIDs
2021-07-22 12:05:53,065 - openpathsampling.experimental.simstore.sql_backend - DEBUG - New block of 1 UUIDs
2021-07-22 12:05:53,069 - openpathsampling.experimental.simstore.sql_backend - DEBUG - Found 1 UUIDs
2021-07-22 12:05:53,070 - openpathsampling.experimental.simstore.storage - DEBUG - Listing all objects to save
2021-07-22 12:05:53,071 - openpathsampling.experimental.simstore.storage - DEBUG - Found 1 objects
2021-07-22 12:05:53,072 - openpathsampling.experimental.simstore.storage - DEBUG - Deproxying proxy objects
2021-07-22 12:05:53,074 - openpathsampling.experimental.simstore.storage - DEBUG - Found 0 objects to deproxy
2021-07-22 12:05:53,077 - openpathsampling.experimental.simstore.storage - DEBUG - Starting to load 0 objects
2021-07-22 12:05:53,078 - openpathsampling.experi

In [61]:
cv_2.local_cache.parent

In [62]:
cv.local_cache.result_dict

{'218455970078904451893013535049010642960': 3.0,
 '218455970078904451893013535049010643004': 4.0,
 '218455970078904451893013535049010643010': 5.0}

In [63]:
storage._sf_handler.canonical_functions

{'218455970078904451893013535049010642996': <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7f85df6cc190>}

In [64]:
print("\n".join([str(getattr(obj, '__uuid__')) for obj in [cv, cv_reloaded, cv_1, cv_2]]))

218455970078904451893013535049010642996
218455970078904451893013535049010642956
218455970078904451893013535049010642996
218455970078904451893013535049010642996


In [65]:
results = storage._sf_handler.canonical_functions[get_uuid(cv)].local_cache.result_dict

In [66]:
assert len(results) == 3

In [67]:
results

{'218455970078904451893013535049010642960': 3.0,
 '218455970078904451893013535049010643004': 4.0,
 '218455970078904451893013535049010643010': 5.0}

In [68]:
# should be Dict[str, List[StorableFunction]]
# dict length 1; the single value in dict is list length 3
storage._sf_handler.all_functions

defaultdict(list,
            {'218455970078904451893013535049010642996': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7f85df6cc190>,
              <openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7f85df6cbd10>]})

In [69]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 3

In [70]:
storage._sf_handler.clear_non_canonical()

In [71]:
storage._sf_handler.all_functions

defaultdict(list,
            {'218455970078904451893013535049010642996': [<openpathsampling.experimental.simstore.storable_functions.StorableFunction at 0x7f85df6cc190>]})

In [72]:
assert len(storage._sf_handler.all_functions) == 1
assert len(list(storage._sf_handler.all_functions.values())[0]) == 1